## Task 3: Data extraction and entity resolution using Python

In [8]:
import pandas as pd
import sqlite3
from sqlite3 import Error

db = '/content/Assignment_1_ADS.db'

In [9]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def run_query(conn, query):
    # Create a cursor
    cur = conn.cursor()
    # Send the query to the database
    cur.execute(query)
    # Extract the results of the query
    results = cur.fetchall()
    # Return the results
    return results

def convert_db_table_to_DF(conn, table):
    # get the names of the attributes in the database table
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
#     print (header_query)
    cols_init = run_query(conn, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
#     print(cols)
    # get the records of the table
    content_query = "Select * from " + table
    data = run_query(conn, content_query)
    df = pd.DataFrame(data, columns = cols)
    return df

#Question B

In [10]:
#todo read customer relation table into panadas df

#connecting to database
sql_db = create_connection(db)

#running queries
print(run_query(sql_db, 'select * from customer'))

#converting db table into pandas dataframe
df_customer = convert_db_table_to_DF(sql_db, 'customer')
print(df_customer)


[('C001', 'John', 'Doe', '1985-03-15', 'john.doe@email.com', '+12345678901', 'Maple Street', '123', 'New York', 'USA', '10001'), ('C002', 'Jane', 'Smith', '1990-07-22', 'jane.smith@email.com', '+12345678902', 'Oak Avenue', '456', 'Los Angeles', 'USA', '90001'), ('C003', 'Michael', 'Johnson', '1978-11-09', 'michael.johnson@email.com', '+12345678903', 'Pine Road', '789', 'Chicago', 'USA', '60601'), ('C004', 'Emily', 'Williams', '1995-02-28', 'emily.williams@email.com', '+12345678904', 'Elm Street', '101', 'Houston', 'USA', '77001'), ('C005', 'David', 'Brown', '1982-08-17', 'david.brown@email.com', '+12345678905', 'Birch Lane', '202', 'Phoenix', 'USA', '85001'), ('C006', 'Sarah', 'Jones', '1998-12-05', 'sarah.jones@email.com', '+12345678906', 'Cedar Court', '303', 'Philadelphia', 'USA', '19101'), ('C007', 'Chris', 'Miller', '1972-04-14', 'chris.miller@email.com', '+12345678907', 'Maple Street', '404', 'San Antonio', 'USA', '78201'), ('C008', 'Jessica', 'Garcia', '1989-09-10', 'jessica.gar

#Question C

Using a similarity function that compares two records (similar to the one in the tutorial), report the customers with similarity > 0.7.

#Jaccard Similarity


In [11]:
# Jaccard similarity function
def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0
#printing customer names from records with Jaccard similarity greater than specified value
similarity_value = 0.7
print('Customers with jaccard similarity greater than ' + str(similarity_value) + ':\n')
for i in range(len(df_customer)):
  record1 = df_customer.iloc[i]
  for j in range(i+1, len(df_customer)):
    record2 = df_customer.iloc[j]
    if jaccard_similarity(set(record1), set(record2)) > similarity_value:
      print(record1.first_name + ' and ' + record2.first_name)

similarity_found = False
# If no similarity was found, print the message
if not similarity_found:
    print(f'No customers with Jaccard similarity greater than {similarity_value} were found.')

Customers with jaccard similarity greater than 0.7:

No customers with Jaccard similarity greater than 0.7 were found.


#Tversky Index

In [12]:
#Tversky Index
def Tversky_Coefficient(set1, set2):
    intersection_size = len(set1.intersection(set2))
    S1_diff_S2 = len(set1.difference(set2))
    S2_diff_S1 = len(set2.difference(set1))
    alpha = 0.2
    beta = 0.8
    denominator = intersection_size + (alpha * S1_diff_S2) + (beta * S2_diff_S1)
    return intersection_size / denominator if denominator != 0 else 0.0

# convert every row to a list and store it to a list
rows = df_customer.values.tolist()

num_rows = len(rows)

# compare each row with each other
for i in range(num_rows):
    for j in range(i+1, num_rows):
        #print(i, j)
        set1 = set(rows[i])
        set2 = set(rows[j])
        similarity = Tversky_Coefficient(set1, set2)

        if similarity > 0.7:
            print(f"Tversky_Coefficient similarity between row {i} and row {j} = ", round(similarity, 3))

Tversky_Coefficient similarity between row 0 and row 15 =  0.818
